In [2]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# image = preprocess(Image.open("CLIP.png")).unsqueeze(0).to(device)
# text = clip.tokenize(["a diagram", "a dog", "a cat"]).to(device)

# with torch.no_grad():
#     image_features = model.encode_image(image)
#     text_features = model.encode_text(text)
    
#     logits_per_image, logits_per_text = model(image, text)
#     probs = logits_per_image.softmax(dim=-1).cpu().numpy()

# print("Label probs:", probs)  # prints: [[0.9927937  0.00421068 0.00299572]]

i = 0
for name, param in model.named_parameters():  # named_parameters 가중치 뽑아주는 함수
    
    print(i,name)
    print(param.shape)
    i+= 1

0 positional_embedding
torch.Size([77, 512])
1 text_projection
torch.Size([512, 512])
2 logit_scale
torch.Size([])
3 visual.class_embedding
torch.Size([768])
4 visual.positional_embedding
torch.Size([50, 768])
5 visual.proj
torch.Size([768, 512])
6 visual.conv1.weight
torch.Size([768, 3, 32, 32])
7 visual.ln_pre.weight
torch.Size([768])
8 visual.ln_pre.bias
torch.Size([768])
9 visual.transformer.resblocks.0.attn.in_proj_weight
torch.Size([2304, 768])
10 visual.transformer.resblocks.0.attn.in_proj_bias
torch.Size([2304])
11 visual.transformer.resblocks.0.attn.out_proj.weight
torch.Size([768, 768])
12 visual.transformer.resblocks.0.attn.out_proj.bias
torch.Size([768])
13 visual.transformer.resblocks.0.ln_1.weight
torch.Size([768])
14 visual.transformer.resblocks.0.ln_1.bias
torch.Size([768])
15 visual.transformer.resblocks.0.mlp.c_fc.weight
torch.Size([3072, 768])
16 visual.transformer.resblocks.0.mlp.c_fc.bias
torch.Size([3072])
17 visual.transformer.resblocks.0.mlp.c_proj.weight
torch.

In [8]:
param[0].dtype

먼저 로라완성 시켜보자.



torch.float32

# PEFT를 이용해서 로라를 완성시킨다


In [ ]:
# !pip install transformers accelerate evaluate datasets peft -q

In [1]:
import transformers
import accelerate
import peft

print(f"Transformers version: {transformers.__version__}")
print(f"Accelerate version: {accelerate.__version__}")
print(f"PEFT version: {peft.__version__}")
"Transformers version: 4.27.4"
"Accelerate version: 0.18.0"
"PEFT version: 0.2.0"

model_checkpoint = "google/vit-base-patch16-224-in21k"

Transformers version: 4.33.0
Accelerate version: 0.24.1
PEFT version: 0.5.0


In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

import torch.nn as nn
import torch.optim as optim
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)
transform = transforms.Compose(
    [transforms.RandomCrop(32, padding=4),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

test_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True) 

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=16,shuffle=False)

/Users/juniverse/opt/anaconda3/envs/veda/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/juniverse/opt/anaconda3/envs/veda/lib/python3.8/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c106detail19maybe_wrap_dim_slowIxEET_S2_S2_b
  Referenced from: <870081F6-12FD-3CEA-BC5C-30F4764F2A98> /Users/juniverse/opt/anaconda3/envs/veda/lib/python3.8/site-packages/torchvision/image.so
  Expected in:     <F2FE5CF8-5B5B-3FAD-ADF8-C77D90F49FC9> /Users/juniverse/opt/anaconda3/envs/veda/lib/python3.8/site-packages/torch/lib/libc10.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


cpu
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
trainset[0][0].shape # image
trainset[0][1] # label

6

In [4]:

labels = trainset.targets

label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label
    break

# id2label[2]
"baklava"

from transformers import AutoImageProcessor

image_processor = AutoImageProcessor.from_pretrained(model_checkpoint)


In [3]:

from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
train_transforms = Compose(
    [
        RandomResizedCrop(image_processor.size["height"]),
        RandomHorizontalFlip(),
        ToTensor(),
        normalize,
    ]
)

val_transforms = Compose(
    [
        Resize(image_processor.size["height"]),
        CenterCrop(image_processor.size["height"]),
        ToTensor(),
        normalize,
    ]
)


def preprocess_train(example_batch):
    """Apply train_transforms across a batch."""
    example_batch["pixel_values"] = [train_transforms(image.convert("RGB")) for image in example_batch["image"]]
    return example_batch


def preprocess_val(example_batch):
    """Apply val_transforms across a batch."""
    example_batch["pixel_values"] = [val_transforms(image.convert("RGB")) for image in example_batch["image"]]
    return example_batch

NameError: name 'image_processor' is not defined

In [ ]:
trainset.set_transform(preprocess_train)
# val_ds.set_transform(preprocess_val)

In [11]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

In [5]:
# Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']

In [6]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

model = AutoModelForImageClassification.from_pretrained(
    model_checkpoint,
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True,  # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
)

pytorch_model.bin:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
x = torch.randn(1,3,224,224)
y = model(x)


print_trainable_parameters(model)


trainable params: 884736 || all params: 86684930 || trainable%: 1.02


In [19]:
for name,params in model.named_parameters():
    print(params.shape)
    print(name)
    

torch.Size([1, 1, 768])
vit.embeddings.cls_token
torch.Size([1, 197, 768])
vit.embeddings.position_embeddings
torch.Size([768, 3, 16, 16])
vit.embeddings.patch_embeddings.projection.weight
torch.Size([768])
vit.embeddings.patch_embeddings.projection.bias
torch.Size([768, 768])
vit.encoder.layer.0.attention.attention.query.weight
torch.Size([768])
vit.encoder.layer.0.attention.attention.query.bias
torch.Size([16, 768])
vit.encoder.layer.0.attention.attention.query.lora_A.default.weight
torch.Size([768, 16])
vit.encoder.layer.0.attention.attention.query.lora_B.default.weight
torch.Size([768, 768])
vit.encoder.layer.0.attention.attention.key.weight
torch.Size([768])
vit.encoder.layer.0.attention.attention.key.bias
torch.Size([16, 768])
vit.encoder.layer.0.attention.attention.key.lora_A.default.weight
torch.Size([768, 16])
vit.encoder.layer.0.attention.attention.key.lora_B.default.weight
torch.Size([768, 768])
vit.encoder.layer.0.attention.attention.value.weight
torch.Size([768])
vit.encod

In [12]:
print_trainable_parameters(model)
"trainable params: 85876325 || all params: 85876325 || trainable%: 100.00"

trainable params: 85799425 || all params: 85799425 || trainable%: 100.00


'trainable params: 85876325 || all params: 85876325 || trainable%: 100.00'

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["query", "value","key"],
    lora_dropout=0.1,
    bias="none",
    modules_to_save=["classifier"],
)
lora_model = get_peft_model(model, config)

print('query value key')
print_trainable_parameters(lora_model)

print('query value')
"trainable params: 667493 || all params: 86466149 || trainable%: 0.77"

trainable params: 884736 || all params: 86684930 || trainable%: 1.02


'trainable params: 667493 || all params: 86466149 || trainable%: 0.77'

In [17]:
for name,params in lora_model.named_parameters():
    if params.requires_grad==True:
        print(name)

base_model.model.vit.encoder.layer.0.attention.attention.query.lora_A.default.weight
base_model.model.vit.encoder.layer.0.attention.attention.query.lora_B.default.weight
base_model.model.vit.encoder.layer.0.attention.attention.key.lora_A.default.weight
base_model.model.vit.encoder.layer.0.attention.attention.key.lora_B.default.weight
base_model.model.vit.encoder.layer.0.attention.attention.value.lora_A.default.weight
base_model.model.vit.encoder.layer.0.attention.attention.value.lora_B.default.weight
base_model.model.vit.encoder.layer.1.attention.attention.query.lora_A.default.weight
base_model.model.vit.encoder.layer.1.attention.attention.query.lora_B.default.weight
base_model.model.vit.encoder.layer.1.attention.attention.key.lora_A.default.weight
base_model.model.vit.encoder.layer.1.attention.attention.key.lora_B.default.weight
base_model.model.vit.encoder.layer.1.attention.attention.value.lora_A.default.weight
base_model.model.vit.encoder.layer.1.attention.attention.value.lora_B.def